In [2]:
from sklearn.metrics import silhouette_score
from pandas.plotting import parallel_coordinates
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns       
from sklearn.decomposition import PCA

In [3]:
data = pd.read_csv('customer_segmentation.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'customer_segmentation.csv'

In [ ]:
data.head()

In [ ]:
dates = ['order_purchase_timestamp', 'order_approved_at', 'order_approved_at', 'order_delivered_carrier_date',
            'order_delivered_customer_date', 'order_estimated_delivery_date', 'shipping_limit_date']

CONVERTING THE VARIABLES INTO DATE BY USING PANDAS:

In [ ]:
for i in dates:
    data[i] = pd.to_datetime(data[i])

In [ ]:
data

WE CAN ALSO DISTINCT DATE FROM TIME:

In [ ]:
data['order_date'] = [d.date() for d in data['order_purchase_timestamp']]
data['order_time'] = [d.time() for d in data['order_purchase_timestamp']]

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.describe()  #Dataset visualization with all the values

In [ ]:
data.columns  #Columns of the dataset 

In [ ]:
data.nunique()  #The output of number of unique values is returned

In [ ]:
data['order_id'].unique()

In [ ]:
data['seller_city'].unique()

In [ ]:
data.isnull().sum()  #The sum of th enull values for each variable

WE CAN REMOVE THE DUPLICATES 

In [ ]:
data.duplicated().sum()
data.drop_duplicates(keep='first', inplace=True)
data.reset_index(drop=True, inplace=True)  #resetting the index of the dataframe

OUTLIERS ANALYSIS

In [ ]:
plt.figure()
data.reset_index().plot(kind='scatter', x='index',
                      y='payment_installments', c='brown')

plt.figure() 
data.reset_index().plot(kind='scatter', x='index', y='payment_value', c='gray')

plt.figure()
data.reset_index().plot(kind='scatter', x='index', y='price', c='orange')

plt.figure()
data.reset_index().plot(kind='scatter', x='index', y='freight_value', c='purple')
plt.figure()

We analyze the 'payment_value' variable because in our mind is one of the most interesting and important variable to study for a big store store like the one in our case. Since it seems having some values that exceed the average, we can suppose that there are some customers who have paid an amount of money which is relatively higher than the other and this can affect our analysis in the future.

In [ ]:
mean_pv = data["payment_value"].mean()  #result mean = 195
std_pv = data["payment_value"].std()    #result Standard Deviation = 295.5
print(mean_pv)
print(std_pv) 

Taking a look at the scatterplot of the outliers, we can notice that there are few payments above (around) 3500$, so we want as an output all the payments above that amount:

In [ ]:
outlier1 = data[data['payment_value'] > 3500]
print('\nOutlier dataframe:\n', outlier1)

#the output will be 5 payments, each one with all its attributes.

We now delete those outliers as they're not relevant for out analysis:

In [ ]:
data = data.drop(data[data.payment_value > 3500].index)
data = data.reset_index(drop=True) 

RELATIONSHOP ANALYSIS:

CORRELATION:

In [ ]:
correlation = data.corr() 

We can print an heatmap matrix in order to visualize clearly the correlation between each couple of variables:

In [ ]:
sns.heatmap(correlation, xticklabels = correlation.columns, yticklabels = correlation.columns, annot = True)

In [ ]:
sns.pairplot(data)  #This method allows us allows us to plot pairwise relationships between variables within a large dataset dataset like ours, so that we can visualize everything in one figure.

In [ ]:
sns.relplot(x = 'order_item_id', y = 'payment_type', hue = 'customer_state', data = data)

In [ ]:
sns.distplot(data['price'])  #Seaborn Distplot represents the overall distribution of continuous data variables.

WHERE DO MOST CUSTOMERS COME FROM?

In [ ]:
state_data = data.groupby('customer_state').count()['customer_id'].reset_index()
city_data = data.groupby('customer_city').count()['customer_id'].reset_index()

plt.figure(figsize = (25,7))

plt.subplot(121)
base_color = sns.color_palette()[0]

sns.barplot(data = state_data.sort_values('customer_id', ascending = False), x = 'customer_state', y = 'customer_id', color = base_color)
plt.title('Number of Customers Per State')
plt.xlabel('State')
plt.ylabel('Number of Customers')

plt.subplot(122)
base_color = sns.color_palette()[0]

sns.barplot(data = city_data.sort_values('customer_id', ascending = False).nlargest(10,'customer_id'), x = 'customer_id', y = 'customer_city', color = base_color)
plt.title('Cities with the Most Customers')
plt.xlabel('City')
plt.ylabel('Number of Customers');

Here we can see that the state with the higher amount of customers is SP, while the city is Sao Paulo

WHAT ARE THE MOST FREQUENT ITEMS BOUGHT?

In [ ]:
best_seller_p = data['product_category_name_english'].value_counts(
).reset_index().nlargest(10, 'product_category_name_english')
worst_seller_p = data['product_category_name_english'].value_counts(
).reset_index().nsmallest(10, 'product_category_name_english')
print(best_seller_p)
print(worst_seller_p)

TOP 10 items bought:

In [ ]:
plt.subplot(211)
sns.barplot(data = best_seller_p, x = 'product_category_name_english',
            y = 'index')
plt.title('Top 10 product Categories Ordered', fontweight='bold')
plt.xlabel('Number of Orders')
plt.ylabel('Product Category')

LOWEST 10 items bought:

In [ ]:
plt.subplot(212)
sns.barplot(data = worst_seller_p, x = 'product_category_name_english',
            y = 'index')
plt.title('Lowest 10 Product Categories Ordered', fontweight = 'bold')
plt.xlabel('Number of Orders')
plt.ylabel('Product Category')

Bed and bath products are the top products ordered followed by beauty products and housewares.

Musical products have the lowest amount of products ordered, followed by flowers and children clothes.

WHICH ARE THE MOST COMMON PAYMENT TYPES?

In [ ]:
payment_types = data['payment_type'].value_counts().reset_index() #count the number of different types of payment
print(payment_types)

WHICH ARE THE NUMBER OF ORDERS PER PAYMENT TYPE?

In [ ]:
plt.subplot(121)
sns.barplot(data = payment_types, x = 'index', y = 'payment_type')
plt.title('Orders by Payment type')
plt.xlabel('Payment Type')
plt.ylabel('Number of Orders');

WHICH IS THE NUMBER OF ORDERS WITH NUMBER OF PAYMENT INSTALLMENTS?

In [ ]:
plt.subplot(122)
sns.barplot(data = data['payment_installments'].value_counts().reset_index(), x = 'index', y = 'payment_installments')
plt.title('Count of Orders With Number of Payment Installments')
plt.xlabel('Number of Payment Installments')
plt.ylabel('Count of Orders');

RFM ANALYSIS:

We start with RECENCY, which is the time since a customer's last purchase. It is calculated by subtracting the customer's last shopping date from each shopping timestamp:

In [ ]:
data['recency'] = print((max(data['order_purchase_timestamp']) - data['order_purchase_timestamp']))

As we only want to know the days since a customer's last purchase, we could have used directly the variable 'order_date' that we initialized at the beginning of the project, but we noticed that it seems to round up the days by considering if the time of the purchase is close to the next day or not, so we are leaving both methods for sake of completeness:

In [ ]:
data['recency'] = (max(data['order_date']) - data['order_date'])

Furthermore, for the customers who made more than one purchase we will only consider the most recent purchase:

In [ ]:
data['recency'] = data.groupby(['customer_unique_id'], as_index=False)['recency'].transform('min')

As a last step we will calculate mean, standard deviation, minimum value and maximum value for recency (we will do the same with the other two components of RFM):

In [ ]:
data['recency'] = data['recency'].dt.days


In [ ]:
print('mean: ', data['recency'].mean())
print('std: ', data['recency'].std())
print('max: ', data['recency'].max())
print('min: ', data['recency'].min())  

FREQUENCY: This is the total number of purchases of the customer. In a different way, it gives the frequency of purchases made by the customer. We just count the number of purchases per customer, obviously considering each customer's unique id as a parameter:

In [ ]:
data['frequency'] = data.groupby(['customer_unique_id'], as_index=False)['order_id'].transform('count')

In [ ]:
print('mean: ', data['frequency'].mean())
print('std: ', data['frequency'].std())
print('max: ', data['frequency'].max())
print('min: ', data['frequency'].min())


MONETARY: It is the total amount of money spent by the customer. It can be calculted by summing all the 'payment_values':

In [ ]:
data['monetary'] = data.groupby(['customer_unique_id'], as_index=False)['payment_value'].transform('sum')

In [ ]:
print('mean: ', data['monetary'].mean())
print('std: ', data['monetary'].std())
print('max: ', data['monetary'].max())
print('min: ', data['monetary'].min())

As you can see in the computation of Frequency and Monetary, the columns of 'order_id' and 'payment_value' are replaced by the result of the computation (sum and count), even if in the putput they might not have the name changed.

We now want to create a first segmentation of the customers based on the RFM. To do this we are going to convert ther RFM scores to a single variable from 1 to 5. We are going to choose the cluster a customer will be assigned to based on the values of mean, std, min and max.

RECENCY

In [ ]:
data['recency_score'] = ''

In [ ]:
for i in data.index:
    if data['recency'][i] <= 20:
        data['recency_score'][i] = 5
    elif (data['recency'][i] > 20) and (data['recency'][i] <= 65): 
        data['recency_score'][i] = 4
    elif (data['recency'][i] > 65) and (data['recency'][i] <= 120):
        data['recency_score'][i] = 3
    elif (data['recency'][i] > 120) and (data['recency'][i] <= 200):
        data['recency_score'][i] = 2
    elif (data['recency'][i] > 200): 
        data['recency_score'][i] = 1  

In this case, talking about recency, we know that the mean is 72 days, so we thought that giving 4 to those customers that haven't bought for 20-65 days was the best option. For the other intervals of time, we kept it quite large considering that the strandard deviation for this score is 42 days.

In [ ]:
data

FREQUENCY

In [ ]:
data['frequency_score'] = ''


In [ ]:
for i in data.index:
    if data['frequency'][i] >= 9:
        data['frequency_score'][i] = 5
    elif (data['frequency'][i] > 7) and (data['frequency'][i] < 9):
        data['frequency_score'][i] = 4
    elif (data['frequency'][i] >= 5) and (data['frequency'][i] < 7):
        data['frequency_score'][i] = 3
    elif (data['frequency'][i] >= 2) and (data['frequency'][i] < 5):
        data['frequency_score'][i] = 2
    elif (data['frequency'][i] == 1):
        data['frequency_score'][i] = 1

In [ ]:
data

With frequency we have been a little bit more 'generous' because, considering that the maximum amount of items bought is 13 and that, on average, the items bought are between 1 anmd 2, we think that a customer who bought more than 9 items can be considered a customer of level 5

MONETARY

In [ ]:
data['monetary_score'] = ''


In [4]:
for i in data.index:
    if data['monetary'][i] > 500:
        data['monetary_score'][i] = 5
    elif (data['monetary'][i] > 250) and (data['monetary'][i] <= 500):
        data['monetary_score'][i] = 4
    elif (data['monetary'][i] > 150) and (data['monetary'][i] <= 250):
        data['monetary_score'][i] = 3
    elif (data['monetary'][i] > 100) and (data['monetary'][i] <= 150):
        data['monetary_score'][i] = 2
    elif (data['monetary'][i] <= 100):
        data['monetary_score'][i] = 1 

NameError: name 'data' is not defined

In [ ]:
data

We are now going to create columns in order to group all the scores. Then, we extracted from the original dataset only the columns of interest so that we can have a better visualization, keeping only the customer id.

In [ ]:
#Recency:
data["recency_score"] = pd.qcut(data['recency'], 5, labels=[5, 4, 3, 2, 1])
#Frequency:
data["frequency_score"] = pd.qcut(data["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
#Monetary:
data["monetary_score"]= pd.qcut(data["monetary"],5,labels=[1,2,3,4,5])

rfm_data = data[[ 'recency', 'frequency', 'monetary', 'recency_score', 'frequency_score', 'monetary_score']]

rfm_data.head()


In [ ]:
rfm = rfm_data[['recency', 'frequency', 'monetary']]

In [ ]:
# Standardizing the features
scaler = StandardScaler()
rfm_std = scaler.fit_transform(rfm)

K Means Clustering

In [ ]:
# Importing libraries for K-means algorithm and performance measurments

# Using elbow method to find out optimal k value with WCSS score
# Creating empty list to insert values
WCSS = []

# Using a for function to fill the list with the WCSS scores
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(rfm_std)
    WCSS.append(kmeans.inertia_)

kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42)
# Use fit_predict to cluster the dataset
km_clusters = kmeans.fit_predict(rfm_std)

rfm_std_cl = pd.DataFrame(rfm_std, columns=rfm.columns, index=rfm.index)
rfm_std_cl['cluster'] = km_clusters





Plotting the Elbow method graph

In [ ]:
# Plotting the Elbow method graph
plt.figure()
plt.plot(range(1, 11), WCSS, marker='o', label='line with marker')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, init='k-means++', random_state=42)
# Use fit_predict to cluster the dataset
km_clusters = kmeans.fit_predict(rfm_std)

rfm_std_cl = pd.DataFrame(rfm_std, columns=rfm.columns, index=rfm.index)
rfm_std_cl['cluster'] = km_clusters

# Visualising the clusters (4 clusters according to the Elbow method)
plt.scatter(rfm_std[km_clusters == 0, 0], rfm_std[km_clusters ==
            0, 1], s=30, c='orange', label='Cluster 1')
plt.scatter(rfm_std[km_clusters == 1, 0], rfm_std[km_clusters ==
            1, 1], s=30, c='yellow', label='Cluster 2')
plt.scatter(rfm_std[km_clusters == 2, 0], rfm_std[km_clusters ==
            2, 1], s=30, c='brown', label='Cluster 3')
plt.scatter(rfm_std[km_clusters == 3, 0], rfm_std[km_clusters ==
            3, 1], s=30, c='purple', label='Cluster 4')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[
            :, 1], s=70, c='black', label='Centroids')
plt.title('Clusters of customers')
plt.legend()
plt.show()

Visualizing the clusters

Calculating cluster validation metrics (silhouette score, calinski harabasz score, davies bouldin score)

In [ ]:
kmeans_s = silhouette_score(rfm_std, kmeans.labels_, metric='euclidean')
kmeans_c = calinski_harabasz_score(rfm_std, kmeans.labels_)
kmeans_d = davies_bouldin_score(rfm_std, km_clusters)
print('Silhouette Score for K-Means:', kmeans_s)
print('Calinski Harabasz Score for K-Means:', kmeans_c)
print('Davies Bouldin Score for K-Means:', kmeans_d)

Hierarchical Clustering

In [ ]:
#Plotting the dendogram
plt.figure()
dendrogram = sch.dendrogram(sch.linkage(rfm_std, method='ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.xticks([])
plt.show()

In [ ]:
# Fitting Hierarchical Clustering to the dataset
hc = AgglomerativeClustering(
    n_clusters=4, affinity='euclidean', linkage='ward')
hc_clusters = hc.fit_predict(rfm_std)

In [ ]:
# Visualising the clusters (4 clusters according to the Dendogram)
plt.figure()
plt.scatter(rfm_std[hc_clusters == 0, 0], rfm_std[hc_clusters == 0, 1], s = 30, c = 'orange', label = 'Cluster 1')
plt.scatter(rfm_std[hc_clusters == 1, 0], rfm_std[hc_clusters == 1, 1], s = 30, c = 'yellow', label = 'Cluster 2')
plt.scatter(rfm_std[hc_clusters == 2, 0], rfm_std[hc_clusters == 2, 1], s = 30, c = 'brown', label = 'Cluster 3')
plt.scatter(rfm_std[hc_clusters == 3, 0], rfm_std[hc_clusters == 3, 1], s = 30, c = 'red', label = 'Cluster 4')
plt.title('Clusters of customers')
plt.legend()
plt.show()

In [ ]:
# Calculating cluster validation metrics
hierarchical_s = silhouette_score(
    rfm_std, hc.labels_, metric='euclidean')
hierarchical_c = calinski_harabasz_score(rfm_std, hc.labels_,)
hierarchical_d = davies_bouldin_score(rfm_std, hc_clusters)
print('Silhouette Score for Hierarchical:', hierarchical_s)
print('Calinski Harabasz Score for Hierarchical:', hierarchical_c)
print('Davies Bouldin Score for Hierarchical:', hierarchical_d)

PCA

In [ ]:
wcss = []
for i in range(1,11):
   model = KMeans(n_clusters = i, init = "k-means++")
   model.fit(rfm_std)
   wcss.append(model.inertia_)
plt.figure(figsize=(10,10))
plt.plot(range(1,11), wcss, marker='o', label='line with marker')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
pca = PCA(3)
data2 = pca.fit_transform(rfm_std)

In [ ]:
plt.figure(figsize=(10,10))
var = np.round(pca.explained_variance_ratio_*100, decimals = 1)
lbls = [str(x) for x in range(1,len(var)+1)]
plt.bar(x=range(1,len(var)+1), height = var, tick_label = lbls)
plt.show()

In [ ]:
model = KMeans(n_clusters = 4, init = "k-means++")
label = model.fit_predict(data2)
plt.figure(figsize=(10,10))
uniq = np.unique(label)
for i in uniq:
   plt.scatter(data2[label == i , 0] , data2[label == i , 1] , label = i)   
plt.scatter(model.cluster_centers_[:, 0], model.cluster_centers_[
            :, 1], s=70, c='black', label='Centroids')
#This is done to find the centroid for each clusters.
plt.legend()
plt.show()

In [ ]:
pca_s = silhouette_score(data2, model.labels_, metric='euclidean')
pca_c = calinski_harabasz_score(data2, model.labels_)
pca_d = davies_bouldin_score(data2, label)
print('Silhouette Score for PCA:', pca_s)
print('Calinski Harabasz Score for PCA:', pca_c)
print('Davies Bouldin Score for PCA:', pca_d)